In [ ]:
!pip install flask-ngrok

In [ ]:
!pip install pandas numpy scikit-fuzzy openpyxl


In [ ]:
import pandas as pd
import numpy as np
import skfuzzy as fuzz


In [ ]:
import pandas as pd
import numpy as np

# 1. Baca data
file_path = "/Blood_samples_data_mentah_missing43.xlsx"
df = pd.read_excel(file_path)

# 2. Imputasi awal dengan mean hanya untuk kolom numerik
numeric_cols = df.select_dtypes(include=[np.number]).columns
data = df.copy()
data[numeric_cols] = data[numeric_cols].fillna(data[numeric_cols].mean())

# 3. Siapkan data_array numerik saja (dan pastikan float)
data_array = data[numeric_cols].to_numpy().T.astype(float)

# 4. Tentukan jumlah cluster (misal 3)
n_clusters = 5

# 5. Jalankan Fuzzy C-Means
import skfuzzy as fuzz
cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(
    data_array, c=n_clusters, m=4, error=0.005, maxiter=1000, init=None
)


In [ ]:
# Asumsi:
# - numeric_cols: list kolom numerik yang ikut FCM
# - cntr: hasil centroid FCM

df_filled = df.copy()
for idx in range(len(df)):
    row = df.iloc[idx]
    missing_cols = row[row.isnull()].index
    if len(missing_cols) > 0:
        # Cari cluster dengan membership tertinggi
        cluster = np.argmax(u[:, idx])
        for col in missing_cols:
            # Pastikan kolom yang hilang adalah kolom numerik (ada di centroid)
            if col in numeric_cols:
                col_idx = numeric_cols.get_loc(col)  # pakai numeric_cols
                df_filled.loc[idx, col] = cntr[cluster, col_idx]


In [ ]:
df_filled.to_excel("Blood_Sample_filled.xlsx", index=False)
print("Nilai hilang berhasil diisi dan file sudah disimpan sebagai 'Blood_Sample_filled.xlsx'")

In [ ]:
from google.colab import files
files.download('Blood_Sample_filled.xlsx')